In [1]:
#load all necessary libraries
import pandas as pd 
import numpy as np 
import scipy as scp
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics 
from sklearn.metrics import confusion_matrix

import statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
pd.set_option("max_columns", 80)

In [3]:
# Read cleaned dataset
df = pd.read_csv('data/interim_data/cdc_data_cleaned_with_ethnicities.csv')
df

,resident_status,level_of_education,month_of_death,age,place_of_death_patient_status,marital_status,day_of_week_of_death,year,manner_of_death,was_autopsy_done,place_of_injury_for_icd_w00_y34_except_y06_and_y07_,icd_code_10th_revision,358_causes_of_death,113_causes_of_death,130_causes_of_infant_death,39_cause_recode,number_of_entity_axis_conditions,entity_condition_1,entity_condition_2,entity_condition_3,number_of_record_axis_conditions,record_condition_1,record_condition_2,record_condition_3,race,race_recode_3,race_recode_5,hispanic_origin,hispanic_originrace_recode,ethnicity
0,RESIDENTS,3.0,January,49,"Hospital, clinic or Medical Center",Divorced,Tuesday,2015,Natural,No,Not Available,O266,All other direct obstetric causes,"Other complications of pregnancy, childbirth a...",Not Available,"Pregnancy, childbirth and the puerperium",9,11O268,21O988,22O992,2,O266,O268,Not Available,White,White,White,Mexican,Mexican,Hispanic
1,RESIDENTS,3.0,January,40,"Hospital, clinic or Medical Center",Married,Sunday,2015,Natural,No,Not Available,O266,All other direct obstetric causes,"Other complications of pregnancy, childbirth a...",Not Available,"Pregnancy, childbirth and the puerperium",6,11O995,21O995,31O266,2,O266,O268,Not Available,White,White,White,Mexican,Mexican,Hispanic
2,RESIDENTS,9.0,April,35,"Hospital, clinic or Medical Center",Married,Friday,2015,Natural,Yes,Not Available,O720,Hemorrhage of pregnancy and childbirth and pla...,"Other complications of pregnancy, childbirth a...",Not Available,"Pregnancy, childbirth and the puerperium",3,11O994,21O720,22O991,1,O720,Not Available,Not Available,White,White,White,Mexican,Mexican,Hispanic
3,RESIDENTS,6.0,April,41,"Hospital, clinic or Medical Center",Married,Wednesday,2015,Natural,No,Not Available,O721,Hemorrhage of pregnancy and childbirth and pla...,"Other complications of pregnancy, childbirth a...",Not Available,"Pregnancy, childbirth and the puerperium",3,11O721,12O991,21O95,1,O721,Not Available,Not Available,White,White,White,Mexican,Mexican,Hispanic
4,INTRASTATE NONRESIDENTS,3.0,June,28,"Hospital, clinic or Medical Center",Married,Sunday,2015,Natural,No,Not Available,O038,Spontaneous abortion,Pregnancy with abortive outcome,Not Available,"Pregnancy, childbirth and the puerperium",8,11O996,12O998,13O995,1,O038,Not Available,Not Available,White,White,White,Mexican,Mexican,Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7130,RESIDENTS,4.0,October,33,"Hospital, clinic or Medical Center",Married,Tuesday,2009,Not Available,No,Not Available,Combination of conditions classifiable to O99....,Indirect obstetric deaths,"Other complications of pregnancy, childbirth a...",Not Available,"Pregnancy, childbirth and the puerperium",1,11O998,Not Available,Not Available,1,O998,Not Available,Not Available,Black,Black,Black,Non – Hispanic,Non - Hispanic black,Black
7131,RESIDENTS,4.0,October,27,"Hospital, clinic or Medical Center",Married,Thursday,2009,Natural,No,Not Available,unspecified pre-eclampsia,Eclampsia and pre-eclampsia,"Other complications of pregnancy, childbirth a...",Not Available,"Pregnancy, childbirth and the puerperium",2,11O149,12O759,Not Available,1,O149,Not Available,Not Available,Black,Black,Black,Non – Hispanic,Non - Hispanic black,Black
7132,RESIDENTS,6.0,December,25,"Hospital, clinic or Medical Center","Never married, Single",Wednesday,2009,Not Available,No,Not Available,"Exhaustion and fatigue, Peripheral neuritis, o...",All other direct obstetric causes,"Other complications of pregnancy, childbirth a...",Not Available,"Pregnancy, childbirth and the puerperium",2,11O268,21O995,Not Available,1,O268,Not Available,Not Available,Black,Black,Black,Non – Hispanic,Non - Hispanic black,Black
7133,RESIDENTS,3.0,December,40,"Hospital, clinic or Medical Center","Never married, Single",Saturday,2009,Natural,Yes,Not Available,Late maternal deaths,"Other deaths related to pregnancy, childbirth ...","Other co

In [4]:
# drop columns
df = df.drop([
    '39_cause_recode', 'entity_condition_2', 'entity_condition_3', 'record_condition_2', 'record_condition_3',
    'race', 'race_recode_3', 'race_recode_5', 'hispanic_origin', 'hispanic_originrace_recode',
    '130_causes_of_infant_death'], axis = 1)

In [5]:
df.columns

Index(['resident_status', 'level_of_education', 'month_of_death', 'age',
       'place_of_death_patient_status', 'marital_status',
       'day_of_week_of_death', 'year', 'manner_of_death', 'was_autopsy_done',
       'place_of_injury_for_icd_w00_y34_except_y06_and_y07_',
       'icd_code_10th_revision', '358_causes_of_death', '113_causes_of_death',
       'number_of_entity_axis_conditions', 'entity_condition_1',
       'number_of_record_axis_conditions', 'record_condition_1', 'ethnicity'],
      dtype='object')

In [6]:
df['marital_status'].unique()

array(['Divorced', 'Married', 'Never married, Single',
       'Marital Status unknown', 'Widowed'], dtype=object)

In [7]:
#Recode values to integer: LinearRegression does not take strings

# def recode_to_int(column_name):
#     df[column_name] = df[column_name].astype('category')
#     df[column_name] = df[column_name].cat.codes

# for column in df.columns:
#     recode_to_int(column)

In [8]:
# remove unknown marital status from analysis
known_marital_status_df = df.loc[df["marital_status"] != "Marital Status unknown"]

In [9]:
df_mlr = known_marital_status_df[['ethnicity', 'marital_status']]
df_mlr

,ethnicity,marital_status
0,Hispanic,Divorced
1,Hispanic,Married
2,Hispanic,Married
3,Hispanic,Married
4,Hispanic,Married
...,...,...
7130,Black,Married
7131,Black,Married
7132,Black,"Never married, Single"
7133,Black,"Never married, Single"


In [10]:
df_mlr['black'] = df_mlr['ethnicity'].apply(lambda x: 1 if x=="Black" else 0)
df_mlr

df_mlr['single'] = df_mlr['marital_status'].apply(lambda x: 1 if x=="Never married, Single" else 0)
df_mlr

<ipython-input-10-3bac07d910ce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mlr['black'] = df_mlr['ethnicity'].apply(lambda x: 1 if x=="Black" else 0)
<ipython-input-10-3bac07d910ce>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mlr['single'] = df_mlr['marital_status'].apply(lambda x: 1 if x=="Never married, Single" else 0)


,ethnicity,marital_status,black,single
0,Hispanic,Divorced,0,0
1,Hispanic,Married,0,0
2,Hispanic,Married,0,0
3,Hispanic,Married,0,0
4,Hispanic,Married,0,0
...,...,...,...,...
7130,Black,Married,1,0
7131,Black,Married,1,0
7132,Black,"Never married, Single",1,1
7133,Black,"Never married, Single",1,1


In [11]:
df_mlr = df_mlr.drop(['ethnicity', 'marital_status'], axis = 1)
df_mlr

,black,single
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
7130,1,0
7131,1,0
7132,1,1
7133,1,1


In [12]:
X = df_mlr['black'].to_frame()
X

,black
0,0
1,0
2,0
3,0
4,0
...,...
7130,1
7131,1
7132,1
7133,1


In [13]:
# variable to predict
y = df_mlr['single']
y

0       0
1       0
2       0
3       0
4       0
       ..
7130    0
7131    0
7132    1
7133    1
7134    0
Name: single, Length: 7053, dtype: int64

In [14]:
print(list(X.columns.values))

#Create training and test datasets
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.20, random_state = 5)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

['black']
(5642, 1)
(1411, 1)
(5642,)
(1411,)


In [15]:
model1 = LogisticRegression(
    random_state=0,
    # multi_class='multinomial'
    multi_class='ovr',
    penalty='none',
    solver='newton-cg'
).fit(X_train, y_train)

In [16]:
def get_log_odds(model):
    log_odds = model.coef_[0]

    return pd.DataFrame(log_odds, 
                        X.columns, 
                        columns=['coef'])\
                        .sort_values(by='coef', ascending=False)

In [17]:
get_log_odds(model1)

,coef
black,1.246391


In [18]:
def exponentiate_log_odds(model):
    odds = np.exp(model.coef_[0])
    return pd.DataFrame(odds,
                        X.columns, 
                        columns=['coef'])\
                        .sort_values(by='coef', ascending=False)

In [19]:
exponentiate_log_odds(model1)

,coef
black,3.477769


In [20]:
preds = model1.predict(X_test)
preds

array([0, 0, 0, ..., 0, 0, 0])

In [21]:
def print_model_params(model):
    params = model.get_params()
    print(params)
    print('Intercept: \n', model.intercept_)
    print('Coefficients: \n', model.coef_)

In [22]:
print_model_params(model1)

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'ovr', 'n_jobs': None, 'penalty': 'none', 'random_state': 0, 'solver': 'newton-cg', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Intercept: 
 [-0.7385109]
Coefficients: 
 [[1.24639101]]


In [23]:
#Calculate odds ratio estimates
np.exp(model1.coef_)

array([[3.47776907]])

In [24]:
def print_regression_results(y_train, X_train):
    logit_model=sm.MNLogit(y_train,sm.add_constant(X_train))
    logit_model
    result=logit_model.fit()
    stats1=result.summary()
    stats2=result.summary2()
    print(stats1)
    print(stats2)

In [25]:
print_regression_results(y_train, X_train)

Optimization terminated successfully.
         Current function value: 0.639800
         Iterations 4
                          MNLogit Regression Results                          
Dep. Variable:                 single   No. Observations:                 5642
Model:                        MNLogit   Df Residuals:                     5640
Method:                           MLE   Df Model:                            1
Date:                Sat, 07 Aug 2021   Pseudo R-squ.:                 0.05947
Time:                        17:29:40   Log-Likelihood:                -3609.8
converged:                       True   LL-Null:                       -3838.0
Covariance Type:            nonrobust   LLR p-value:                2.776e-101
  single=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.7385      0.035    -21.384      0.000      -0.806      -0.671
black          1.2464      0.

In [26]:
# To check accuracy
# Create a confusion matrix
confusion_matrix(y_test, preds)

#transform confusion matrix into array
# confmtrx = np.array(confusion_matrix(y_test, preds))

array([[631, 169],
       [336, 275]])

In [27]:
def print_accuracy_statistics(y_test, preds):
    print('Accuracy Score:', metrics.accuracy_score(y_test, preds))
    
    #Create classification report
    class_report=classification_report(y_test, preds)
    print(class_report)

In [28]:
print_accuracy_statistics(y_test, preds)

Accuracy Score: 0.6420978029766123
              precision    recall  f1-score   support

           0       0.65      0.79      0.71       800
           1       0.62      0.45      0.52       611

    accuracy                           0.64      1411
   macro avg       0.64      0.62      0.62      1411
weighted avg       0.64      0.64      0.63      1411



---

In [29]:
df['358_causes_of_death'].unique()

array(['All other direct obstetric causes',
       'Hemorrhage of pregnancy and childbirth and placenta previa',
       'Spontaneous abortion',
       'Other deaths related to pregnancy, childbirth and the puerperium',
       'Ectopic pregnancy', 'Obstetric embolism',
       'Obstetric death of unspecified cause',
       'Eclampsia and pre-eclampsia', 'Indirect obstetric deaths',
       'Other complications predominately related to the puerperium',
       'Other and unspecified pregnancy with abortive outcome',
       'Other abortion', 'Medical abortion'], dtype=object)

In [30]:
df_mlr = df[['ethnicity', '358_causes_of_death']]
df_mlr

,ethnicity,358_causes_of_death
0,Hispanic,All other direct obstetric causes
1,Hispanic,All other direct obstetric causes
2,Hispanic,Hemorrhage of pregnancy and childbirth and pla...
3,Hispanic,Hemorrhage of pregnancy and childbirth and pla...
4,Hispanic,Spontaneous abortion
...,...,...
7130,Black,Indirect obstetric deaths
7131,Black,Eclampsia and pre-eclampsia
7132,Black,All other direct obstetric causes
7133,Black,"Other deaths related to pregnancy, childbirth ..."


In [31]:
df_mlr['black'] = df_mlr['ethnicity'].apply(lambda x: 1 if x=="Black" else 0)

df_mlr['eclampsia_and_pre_eclampsia'] = df_mlr['358_causes_of_death'].apply(lambda x: 1 if x=="Eclampsia and pre-eclampsia" else 0)

df_mlr = df_mlr.drop(['ethnicity', '358_causes_of_death'], axis = 1)

X = df_mlr['black'].to_frame()
y = df_mlr['eclampsia_and_pre_eclampsia']

print(list(X.columns.values))

#Create training and test datasets
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.20, random_state = 5)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

['black']
(5708, 1)
(1427, 1)
(5708,)
(1427,)


<ipython-input-31-f23649e154b4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mlr['black'] = df_mlr['ethnicity'].apply(lambda x: 1 if x=="Black" else 0)
<ipython-input-31-f23649e154b4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mlr['eclampsia_and_pre_eclampsia'] = df_mlr['358_causes_of_death'].apply(lambda x: 1 if x=="Eclampsia and pre-eclampsia" else 0)


In [32]:
df_mlr

,black,eclampsia_and_pre_eclampsia
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
7130,1,0
7131,1,1
7132,1,0
7133,1,0


In [33]:
model2 = LogisticRegression(
    random_state=0,
    # multi_class='multinomial'
    multi_class='ovr',
    penalty='none',
    solver='newton-cg'
).fit(X_train, y_train)

In [34]:
get_log_odds(model2)

,coef
black,0.469017


In [35]:
exponentiate_log_odds(model2)

,coef
black,1.598422


In [36]:
preds = model2.predict(X_test)

In [37]:
print_model_params(model2)

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'ovr', 'n_jobs': None, 'penalty': 'none', 'random_state': 0, 'solver': 'newton-cg', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Intercept: 
 [-3.15086461]
Coefficients: 
 [[0.46901677]]


In [38]:
np.exp(model2.coef_)

array([[1.5984218]])

In [39]:
print_regression_results(y_train, X_train)

Optimization terminated successfully.
         Current function value: 0.192448
         Iterations 7
                               MNLogit Regression Results                              
Dep. Variable:     eclampsia_and_pre_eclampsia   No. Observations:                 5708
Model:                                 MNLogit   Df Residuals:                     5706
Method:                                    MLE   Df Model:                            1
Date:                         Sat, 07 Aug 2021   Pseudo R-squ.:                0.006144
Time:                                 17:29:40   Log-Likelihood:                -1098.5
converged:                                True   LL-Null:                       -1105.3
Covariance Type:                     nonrobust   LLR p-value:                 0.0002284
eclampsia_and_pre_eclampsia=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------

In [40]:
confusion_matrix(y_test, preds)

array([[1356,    0],
       [  71,    0]])

In [41]:
print_accuracy_statistics(y_test, preds)

Accuracy Score: 0.9502452697967765
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      1356
           1       0.00      0.00      0.00        71

    accuracy                           0.95      1427
   macro avg       0.48      0.50      0.49      1427
weighted avg       0.90      0.95      0.93      1427



/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
